In [0]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [2]:
# Root directory of the project
ROOT_DIR = os.path.abspath("./")
print('SET ROOT DIR:', ROOT_DIR)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log


from Mask_RCNN.samples.nucleus import nucleus

# Directory to save logs and trained model
LOGS_DIR = os.path.join(ROOT_DIR, "logs")

DEVICE = "/cpu:0" # or /gpu:0
TEST_MODE = "inference"
DATASET_DIR = os.path.join(ROOT_DIR, "Mask_RCNN/nuclei_datasets")

SET ROOT DIR: /content


Using TensorFlow backend.


In [0]:
def compute_batch_ap(dataset, image_ids, verbose=1):
    APs = []
    for image_id in image_ids:
        # Load image
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config,
                                   image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect_molded(image[np.newaxis], image_meta[np.newaxis], verbose=0)
        # Compute AP over range 0.5 to 0.95
        r = results[0]
        ap = utils.compute_ap_range(
            gt_bbox, gt_class_id, gt_mask,
            r['rois'], r['class_ids'], r['scores'], r['masks'],
            verbose=0)
        APs.append(ap)
        if verbose:
            info = dataset.image_info[image_id]
            meta = modellib.parse_image_meta(image_meta[np.newaxis,...])
            print("{:3} {}   AP: {:.2f}".format(
                meta["image_id"][0], meta["original_image_shape"][0], ap))
    return APs

In [4]:
# Inference Configuration
config = nucleus.NucleusInferenceConfig()

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference",
                              model_dir=LOGS_DIR,
                              config=config)

weights_path = os.path.join(ROOT_DIR, "Mask_RCNN/weights/kaggle_bowl.h5")

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

W0627 04:27:31.599661 140552534747008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0627 04:27:31.608189 140552534747008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0627 04:27:31.614920 140552534747008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0627 04:27:31.649165 140552534747008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1919: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0627 04:27:31.654630 140552534

Loading weights  /content/Mask_RCNN/weights/kaggle_bowl.h5


## Let ready datasets

In [5]:
# Load validation dataset
train_dataset = nucleus.NucleusDataset()
train_dataset.load_nucleus(DATASET_DIR, "train")
train_dataset.prepare()

print("Images: {}\nClasses: {}\n".format(len(train_dataset.image_ids), train_dataset.class_names))

# Load validation dataset
val_dataset = nucleus.NucleusDataset()
val_dataset.load_nucleus(DATASET_DIR, "val")
val_dataset.prepare()

print("\nImages: {}\nClasses: {}".format(len(val_dataset.image_ids), val_dataset.class_names))

Images: 645
Classes: ['BG', 'nucleus']


Images: 25
Classes: ['BG', 'nucleus']


## maP on Validation dataset

only on 5 images

In [6]:
# Run on validation set
limit = 5
APs = compute_batch_ap(val_dataset, val_dataset.image_ids[-limit:]) # last 5 images
print("Mean AP overa {} images: {:.4f}".format(len(APs), np.mean(APs)))

 20 [256 320   3]   AP: 0.41
 21 [256 256   3]   AP: 0.43
 22 [256 320   3]   AP: 0.36
 23 [256 256   3]   AP: 0.86
 24 [256 256   3]   AP: 0.82
Mean AP overa 5 images: 0.5764


## maP on Traing dataset

only on 5 images

In [8]:
# Run on training set
limit = 5
APs = compute_batch_ap(train_dataset, train_dataset.image_ids[-limit:]) # last 5 images
print("Mean AP overa {} images: {:.4f}".format(len(APs), np.mean(APs)))

640 [256 256   3]   AP: 0.87
641 [256 256   3]   AP: 0.58
642 [256 320   3]   AP: 0.41
643 [256 256   3]   AP: 0.66
644 [256 256   3]   AP: 0.35
Mean AP overa 5 images: 0.5740
